# Membrane-Protein MD Setup Jupyter Notebook with BioBB Example

## Using prepared systems from MemProtMD (http://memprotmd.bioch.ox.ac.uk/)

**MemProtMD is a database of around 3500 intrinsic membrane protein structures identified in the Protein Data Bank, inserted into simulated lipid bilayers using Coarse-Grained Self Assembly Molecular Dynamics simulations.**

*Newport, Thomas D. et al. The MemProtMD database: a resource for membrane-embedded protein structures and their lipid interactions. Nucleic Acids Research, Volume 47, Issue D1, 08 January 2019, Pages D390–D397, https://doi.org/10.1093/nar/gky1047*

***

## Settings

### Biobb modules used

* [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
* [biobb_md](https://github.com/bioexcel/biobb_md): Tools to setup and run Molecular Dynamics simulations.
* [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 
### Auxiliar libraries used

* [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
* [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
* [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
* [os](https://docs.python.org/3/library/os.html): Python miscellaneous operating system interfaces
* [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
* [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_md_setup_membrane.git
cd biobb_wf_md_setup_membrane
conda env create -f conda_env/environment.yml
conda activate biobb_MDsetup_Membrane_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_md_setup_membrane/notebooks/Membrane-Protein-MD-Setup.ipynb
```

***
## Pipeline steps
 1. [Select the membrane-protein complex](#select)
 2. [Download system](#download)
 3. [Preparing files](#prep)
 4. [Energetically minimize the system](#min)
 5. [Equilibrate the system (NPT)](#npt)
 6. [Production Molecular Dynamics Simulation](#prod)
 7. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 8. [Output Files](#output)
 9. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

<a id="select"></a>
## Select the membrane-protein complex 

In [ ]:
import json
import os
import pathlib
import zipfile
import nglview

WORKDIR = pathlib.Path.cwd()

In [ ]:
# TO REMOVE!!!!!!
#WORKDIR = '/YOUR/WORKING/DIR'
#os.chdir(WORKDIR)
#print(os.getcwd())

### Full list

#### Getting full list of simulations

**Using MemProtMD REST API to download membrane-proteins prepared systems to start a MD simulation.**

**Example 1:** Getting the **full list of prepared systems** from the **MemProtMD database**.

In [ ]:
from biobb_io.api.memprotmd_sim_list import memprotmd_sim_list

# Create prop dict and inputs/outputs
simulations_list = 'simulations_list.json'

# Create and launch bb
memprotmd_sim_list(output_simulations=simulations_list)

#### Select the membrane-protein complex from the list of available systems 

**Select one** of the available systems from the list and **jump to the ["Download System" cell](#Download-system).**

In [ ]:
import ipywidgets as widgets

# parse simulations_list file
with open(simulations_list) as json_file:
    listJson = json.load(json_file)
    listMDs = []
    for sim in listJson:
        listMDs.append(sim['_id'])
        
# show dropdown with simulations_list file content
mdsel = widgets.Dropdown(
    options=listMDs,
    description='MDs:',
    disabled=False,
)
display(mdsel)

### Particular family

#### Getting particular family of simulations

**Using MemProtMD REST API to download membrane-proteins prepared systems to start a MD simulation.**

**Example 2:** Getting a **list of prepared systems** for a **particular family** (e.g. GPCRs, porins) from the **MemProtMD database**.

In [ ]:
from biobb_io.api.memprotmd_sim_search import memprotmd_sim_search

# Create prop dict and inputs/outputs
simulations_search = 'simulations_search.json'

prop = {
    'collection_name': 'refs',
    'keyword': 'gpcr'
}

# Create and launch bb
memprotmd_sim_search(output_simulations=simulations_search,
                  properties=prop)

#### Select the membrane-protein complex from the list of available systems 

**Select one** of the available systems from the list and **jump to the ["Download System" cell](#Download-system).**

In [ ]:
import ipywidgets as widgets

# parse simulations_list file
with open(simulations_search) as json_file:
    listJson = json.load(json_file)
    listGPCRs = []
    for sim in listJson:
        listGPCRs.append(sim['simulations'][0])
        
# show dropdown with simulations_list file content
mdsel = widgets.Dropdown(
    options=listGPCRs,
    description='GPCRs:',
    disabled=False,
)
display(mdsel)

##############################
##############################
# @Adam: try with 2i37
##############################
##############################


***
<a id="download"></a>
## Download system

Download the **membrane-protein system** and associated **GROMACS files** needed to run the **MD setup** from the **MemProtMD REST API**, using the system we **previously selected**.

In [ ]:
pdbCode = mdsel.value[:4]

from biobb_io.api.memprotmd_sim import memprotmd_sim

# Create prop dict and inputs/outputs
simulation = 'simulation.zip'

prop = {
    'pdb_code': pdbCode
}

# Create and launch bb
memprotmd_sim(output_simulation=simulation,
                  properties=prop)

Create a folder where to unzip the content of simulation.

In [ ]:
# create folder simulation
if not pathlib.Path(pdbCode).exists():
    pathlib.Path(pdbCode).mkdir()
    
# extract simulation into simulation folder
os.chdir(pdbCode)
sim_path = pathlib.PurePosixPath(WORKDIR).joinpath(simulation)

with zipfile.ZipFile(sim_path, 'r') as zip_f:
    zip_f.extractall()

***
<a id="prep"></a>
## Preparing files

Preparing the files from the downloaded zip. Identifying:
* **Membrane-Protein complex atomistic system**: 
    * *atomistic-system.pdb*
* **GROMACS index file**: 
    * *atomistic-system.ndx*
* **Topology files**: 
    * *topol.top* + auxiliar *\*.itp* files
    * We are generating a **single topology file** compressing all these files into **one single zip** file named *{pdbCode}-top.zip*
* **GROMACS Molecular Dynamics Properties (mdp) files**:
    * *em.mdp*: Energy minimization mdp file
    * *pr.mdp*: Equilibration mdp file
    * *100ns.mdp*: Final 100ns MD simulation mdp file

In [ ]:
# Important files
pdb = "atomistic-system.pdb"
top = pdbCode + "-top.zip"
ndx = "atomistic-system.ndx"
mdp_em = "mdp_files/em.mdp"
mdp_pr = "mdp_files/pr.mdp"
mdp_100ns = "mdp_files/100ns.mdp"

# Generate TOP zip file
zf = zipfile.ZipFile(top, mode='w')

for file in os.listdir(os.getcwd()):
    if file.endswith(".top") or file.endswith(".itp") or (file == 'charmm36.ff' and pathlib.Path(file).is_dir()) or (file == 'itp' and pathlib.Path(file).is_dir()):
        if not pathlib.Path(file).is_dir():
            zf.write(file)
        else:
            for dirname, subdirs, files in os.walk(file):
                for filename in files:
                    zf.write(pathlib.PurePosixPath(dirname).joinpath(filename))

zf.close()

### Visualizing 3D structure

Visualizing the **Membrane-Protein complex structure** using **NGL**:

In [ ]:
# Show Membrane Protein
view = nglview.show_structure_file(pdb)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='solute', color='sstruc')
view.add_representation(repr_type='ball+stick', selection='DPPC', color='element', opacity=0.4)
view.add_representation(repr_type='line', radius=1, selection='SOL', opacity=0.4)
view.add_representation(repr_type='spacefill', radius=1, selection='NA')
view.add_representation(repr_type='spacefill', radius=1, selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

**Note** that if you are using **Google Chrome** there can be some memory problems visualizing this structure depending on your computer resorces. If you are experiencing this issue we strongly recommend you to open this notebook in **Mozilla Firefox**.

***
<a id="min"></a>
## Energetically minimize the system
Energetically minimize the **protein system** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **system** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
The **em.mdp** file (**molecular dynamics parameters (mdp))** contains the MemProtMD default parameters to run an **energy minimization** to the prepared Membrane-Protein complex systems:

-  integrator  = steep ; Algorithm (steep = steepest descent minimization)
-  emtol       = 1000.0 ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
-  emstep      = 0.001 ; Minimization step size (nm)
-  nsteps      = 5000 ; Maximum number of (minimization) steps to perform

In this particular example, the method used to run the **energy minimization** is the default **steepest descent**, ¡ the **maximum force** is placed at **1000 KJ/mol\*nm^2**, and the **maximum number of steps** to perform (if the maximum force is not reached) to **5000 steps**. 

In [ ]:
from biobb_md.gromacs.grompp import grompp

# executing Grompp in the simulation folder, though the output file will be stored in the WORKDIR
# Create prop dict and inputs/outputs
output_gppmin_tpr = str(WORKDIR) + "/" + pdbCode+'_gppmin.tpr'

prop = {
    'mdp':{
        'emtol':'500',
        #'nsteps':'5000'
        'nsteps':'500'
    },
    'simulation_type':'minimization'
}

# Create and launch bb
grompp(input_gro_path=pdb, 
       input_top_zip_path=top, 
       #input_mdp_path=mdp_em,
       input_ndx_path= ndx,
       output_tpr_path=output_gppmin_tpr,  
       properties=prop)

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step. 

In [ ]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import mdrun

# execute Mdrun in the WORKDIR
os.chdir(WORKDIR)

# Create prop dict and inputs/outputs
output_min_trr = pdbCode+'_min.trr'
output_min_gro = pdbCode+'_min.gro'
output_min_edr = pdbCode+'_min.edr'
output_min_log = pdbCode+'_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr, 
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro, 
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log)

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [ ]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbCode+'_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop)

In [ ]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
}

plotly.offline.iplot(fig)

***
<a id="npt"></a>
## Equilibrate the system (NPT)
Equilibrate the **protein system** in **NPT** ensemble (constant Number of particles, Pressure and Temperature).
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein system** with **NPT** ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

The **pr.mdp** **(molecular dynamics parameters (mdp))** contains the MemProtMD default parameters to run an **NPT equilibration** with **protein restraints** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  Define                   = -DPOSRES
-  integrator               = md
-  dt                       = 0.002
-  nsteps                   = 500000
-  pcoupl = Parrinello-Rahman
-  pcoupltype = semiisotropic
-  tau_p = 1.0
-  ref_p = 1.0 1.0
-  compressibility = 4.5e-5 4.5e-5
-  gen_vel = yes

In this particular example, the MemProtMD parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, **500,000 equilibration steps** with the protein **heavy atoms restrained**, and a Parrinello-Rahman **pressure coupling** algorithm.

In [ ]:
from biobb_md.gromacs.grompp import grompp

# executing Grompp in the simulation folder, though the output file will be stored in the WORKDIR
os.chdir(pdbCode)
# Create prop dict and inputs/outputs
output_gppeq_tpr = str(WORKDIR) + "/" + pdbCode+'_gppeq.tpr'

prop = {
    'mdp':{
        'nsteps':'500'
        #'nsteps':'5000'
    }
}

# Create and launch bb
grompp(input_gro_path= str(pathlib.PurePosixPath(WORKDIR).joinpath(output_min_gro)),
       input_top_zip_path=top, 
       input_mdp_path= mdp_pr,
       input_ndx_path= ndx,
       output_tpr_path=output_gppeq_tpr,  
       properties=prop)

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

*Please note that because of the simulated time (1ns) and the size of the Membrane-Protein complex, this step may take hours in a usual laptop.*

In [ ]:
# Mdrun: Running Equilibration NPT
from biobb_md.gromacs.mdrun import mdrun

# execute Mdrun in the WORKDIR
os.chdir(WORKDIR)

# Create prop dict and inputs/outputs
output_eq_trr = pdbCode+'_eq.trr'
output_eq_gro = pdbCode+'_eq.gro'
output_eq_edr = pdbCode+'_eq.edr'
output_eq_log = pdbCode+'_eq.log'
output_eq_cpt = pdbCode+'_eq.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppeq_tpr, 
      output_trr_path=output_eq_trr, 
      output_gro_path=output_eq_gro, 
      output_edr_path=output_eq_edr, 
      output_log_path=output_eq_log, 
      output_cpt_path=output_eq_cpt)

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [ ]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_eq_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

In [ ]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

***
<a id="prod"></a>
## Production Molecular Dynamics Simulation
Upon completion of the **equilibration phase**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released, and the **production MD** can be started.

- [Step 1](#mdStep1): Creating portable binary run file to run a **production MD simulation**.
- [Step 2](#mdStep2): Run 100ns MD simulation of the **Membrane-Protein complex system**.
- [Step 3](#mdStep3): Checking results for the **production MD simulation**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms** 
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr** 
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run the production MD simulation

The **100ns.mdp** **(molecular dynamics parameters (mdp))** contains the MemProtMD default parameters to run the **production MD simulation** (see [GROMACS mdp options](http://manual.gromacs.org/documentation/2018/user-guide/mdp-options.html)):

-  integrator               = md
-  dt                       = 0.002 (ps)
-  nsteps                   = 50000000

In this particular example, the MemProtMD default parameters will be used: **md** integrator algorithm, a **time step** of **2fs**, and a total of **5,000,000 MD steps** (100ns).

In [ ]:
from biobb_md.gromacs.grompp import grompp

# executing Grompp in the simulation folder, though the output file will be stored in the WORKDIR
os.chdir(pdbCode)
# Create prop dict and inputs/outputs
output_gppMD_tpr = str(WORKDIR) + "/" + pdbCode+'_gppMD.tpr'

# modifying parameters of mdp_100ns
prop = {
    'mdp':{
        'nsteps':'5000',
        'nstxout':'500',
        'nstvout':'500'
    }
}

# Create and launch bb
grompp(input_gro_path=str(pathlib.PurePosixPath(WORKDIR).joinpath(output_eq_gro)), 
       input_top_zip_path=top, 
       input_mdp_path= mdp_100ns,
       input_ndx_path= ndx,
       output_tpr_path=output_gppMD_tpr,  
       properties=prop)

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

**WARNING:** *Please note that because of the simulated time (100ns) and the size of the Membrane-Protein complex, this step may take serveral hours (even days!) in a usual laptop.*

In [ ]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import mdrun

# execute Mdrun in the WORKDIR
os.chdir(WORKDIR)

# Create prop dict and inputs/outputs
output_md_trr = pdbCode+'_md.trr'
output_md_gro = pdbCode+'_md.gro'
output_md_edr = pdbCode+'_md.edr'
output_md_log = pdbCode+'_md.log'
output_md_cpt = pdbCode+'_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppMD_tpr, 
      output_trr_path=output_md_trr, 
      output_gro_path=output_md_gro, 
      output_edr_path=output_md_edr, 
      output_log_path=output_md_log, 
      output_cpt_path=output_md_cpt)

<a id="mdStep3"></a>
### Step 3: Checking MD simulation results
Checking final results. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **production MD run**. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppMD_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop)

In [ ]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone',
    #'selection': 'non-Water'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp, 
          properties=prop)

In [ ]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = {
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
}

plotly.offline.iplot(fig)


In [ ]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr, 
          properties=prop)

***
<a id="post"></a>
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **Membrane-Protein complex system resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image** 
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str** 
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [ ]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein',
    'output_selection': 'Protein',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image(input_traj_path=output_md_trr,
         input_top_path=output_gppMD_tpr,
         output_traj_path=output_imaged_traj, 
          properties=prop)

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [ ]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = pdbCode+'_md_dry.gro'
prop = {
    'selection':  'Protein'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_md_gro,
         input_top_path=output_gppMD_tpr,
         output_str_path=output_dry_gro, 
          properties=prop)

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [ ]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
